In [66]:
import os
import cv2
import json
import numpy as np

Местоположение файла "crop_traffic_lights.ipynb", когда я его вызывал: '/home/solomentsev_yaroslav/Jupyter_Notebook'
Все дальнейшие условные пути в коде указаны относительно вышеприведенной директории.
Местоположение исходного датасета NAMI-Day-Night: '/home/solomentsev_yaroslav/Jupyter_Notebook/datasets/NAMI-Day-Night'
Местоположение исходного датасета NKBVS_mixed: '/home/solomentsev_yaroslav/Jupyter_Notebook/datasets/NKBVS_mixed'
Местоположение аннотаций json: '/home/solomentsev_yaroslav/Jupyter_Notebook'

In [74]:
json_filename = '99_Taganrog-Day-Night.json'
with open(json_filename) as json_file:
    json_data = json.load(json_file)
    json_file.close()

In [75]:
output_dir = 'datasets/Cropped_traffic_lights_TAGANROG' # Это путь, куда будут сохраняться кропнутые файл.
# В этой же папке создадутся папки с наименованиями цветов светофоров: 'G', 'NA', 'R', 'Y', 'YR'
if not(os.path.isdir(output_dir)):
    os.mkdir(output_dir)

In [76]:
# Здесь устраняются косяки с аннотациями цветов светофоров. Например, когда отметили 'NAR', вместо положенного 'R'
for i, record in enumerate(json_data['annotations']):
    if record['conf'] == 'NAR':
        record['conf'] = 'R'
        json_data['annotations'][i] = record
    elif record['conf'] == 'RNA':
        record['conf'] = 'R'
        json_data['annotations'][i] = record
    elif record['conf'] == 'GY':
        record['conf'] = 'Y'
        json_data['annotations'][i] = record
    elif record['conf'] == 'RN':
        record['conf'] = 'R'
        json_data['annotations'][i] = record

In [77]:
for record_image in json_data['images']:
    file_name = record_image['file_name'] # В file_name помимо самого имени файла также включаются папки. 
    # Для понимания вызывай команду json_data['images']
    image = cv2.imread('datasets/'+file_name)
    file_name = file_name.split('/')[-1] # Получаем наименование файла без имени папок к нему
    list_of_bboxes = [record_annot['bbox'] for record_annot in json_data['annotations'] 
                      if record_annot['image_id'] == record_image['id']] # здесь получаем список всех боксов для данного изображения
    list_of_confs = [record_annot['conf'] for record_annot in json_data['annotations'] # здесь получаем список цветов светофоров для данного изображения для соотвествующих боксов.Цвет светофора хранится в поле "conf"
                      if record_annot['image_id'] == record_image['id']]
    if list_of_bboxes != []:
        for i,box in enumerate(list_of_bboxes):
            x1, y1, width, height = box[0], box[1], box[2], box[3]
            x2 = x1 + width
            y2 = y1 + height
            # x_left, x_right, y_top, y_bottom - это размеры в пикселях, которое нужно взять вокруг светофора. 
            # x_left захватывает область левее светофора, y_top захватывает область выше светофора и т.д. 
            #x_left, x_right, y_top, y_bottom = width / 2, width/2, height / 2, height / 2 # Здесь я делал размер захвата области вокруг светофора равнм половине ширине или высоте светофора 
            x_left, x_right, y_top, y_bottom = 0,0,0,0 # Изображения, находящиеся в папке /datasets/Traffic_lights получены при отсутствии захвата окружающей области светофора, т.е. при нулевых данных параметрах
            x1, x2, y1, y2 = max(x1-x_left,0), min(x2+x_right,image.shape[1]), max(y1-y_top,0), min(y2+y_bottom,image.shape[0]) 
            x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)
            cropped_image = image[y1:y2+1, x1:x2+1]
            intermediate_dir = list_of_confs[i] # intermediate_dir - Папка с именем цвета светофора
            if json_filename == '97_NAMI-Polygon-Day-Night_25_11_2019.json':
                day_night = record_image['file_name'].split('/')[-2] # у датасета 97_NAMI-Polygon-Day-Night есть разделение на Day и Night. Изображения хранятся в соответствующих папках "Day" и "Night". 
            #Используем наименование папки, в которой хранится файл (Day или Night) для того, чтобы поместить 'Day' или 'Night' в имя выходного файла, см. дальше
            else:
                day_night = '' # В других датасетах, например, в NKBVS_mixed, не раделения папок на Day или Night
            if not(os.path.isdir(output_dir+'/'+intermediate_dir)):
                os.mkdir(output_dir+'/'+intermediate_dir)
            cv2.imwrite(output_dir+'/'+intermediate_dir+'/'+day_night+'_'+file_name[:file_name.rfind('.')]+'_'+str(i)+file_name[file_name.rfind('.'):],cropped_image)
            # Здесь file_name.rfind('.') означает найти позицию с конца имени файла, где есть точка. Таким образом я могу выделить имя файла без '.png' (file_name[:file_name.rfind('.')]+) или наоборот выделить только расширение файла без его имени (file_name[file_name.rfind('.'):])